In [1]:
import numpy as np
import tensorflow as tf

In [27]:
input_step=25 #to change after to 10,10
output_step=50

In [28]:
X1=np.load("X1_"+str(input_step)+".npy")
X2=np.load("X2_"+str(input_step)+".npy")

In [29]:
X1.shape

(52316, 25, 6, 500)

In [30]:
X2.shape

(51922, 25, 6, 500)

In [31]:
X1=np.float32(X1)
X2=np.float32(X2)


In [32]:
from keras.layers import Input, Dense, Reshape, Flatten,Dropout
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


input_shape = (input_step,6,500)
hidden_size1 = 256
hidden_size2 = 128
hidden_size3 = 100
code_size = 64

input_layer = Input(shape=(input_shape))
print(input_layer)
hidden_layer_0=Flatten()(input_layer)
hidden_layer_1 = Dense(hidden_size1, activation='relu')(hidden_layer_0)
hidden_layer_2 = Dense(hidden_size2, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(hidden_size3, activation='relu')(hidden_layer_2)
bottleneck_layer = Dense(code_size*input_step, activation='sigmoid')(hidden_layer_3)

# Define the decoder network
hidden_layer_5 = Dense(hidden_size1, activation='relu')(bottleneck_layer)
hidden_layer_6 = Dense(hidden_size2, activation='relu')(hidden_layer_5)
hidden_layer_7 = Dense(hidden_size1, activation='relu')(hidden_layer_6)
hidden_layer_8 = Dense(6*500*input_step, activation='sigmoid')(hidden_layer_7)
output_layer=Reshape((input_step,6,500))(hidden_layer_8)

autoencoder = Model(input_layer, output_layer)
autoencoder.summary()
opt = Adam(learning_rate=0.0001)
autoencoder.compile(optimizer=opt, loss='mae',metrics=['mae', 'mse',tf.keras.metrics.RootMeanSquaredError()])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
autoencoder.fit(X1, X1, epochs=300,batch_size=64,
                validation_data=(X2, X2),callbacks=[early_stopping]
              )

KerasTensor(type_spec=TensorSpec(shape=(None, 25, 6, 500), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 25, 6, 500)]      0         
                                                                 
 flatten_1 (Flatten)         (None, 75000)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               19200256  
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dense_10 (Dense)            (None, 100)               12900     
                                                                 
 dense_11 (Dense)            (None, 160

In [33]:
encoder = Model(input_layer, bottleneck_layer)
encoder.compile(optimizer='adam', loss='mae')

In [34]:
X11=encoder.predict(X1)
X22=encoder.predict(X2)

1623/1623 [==============================] - 21s 13ms/step


In [ ]:
decoder = Model(bottleneck_layer, output_layer)
decoder.compile(optimizer='adam', loss='mae')

In [ ]:
Y1=decoder.predict(X11)
Y2=decoder.predict(X22)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Y1[0][0][5])
plt.plot(X1[0][0][5])

In [ ]:
np.save("X11_"+str(input_step),X11)
np.save("X22_"+str(input_step),X22)

In [26]:
encoder.save("encoder-simple_"+str(input_step)+".h5")

# Cnn encoder

In [49]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D,Dropout
from keras.models import Model

# Define input shape
input_shape = (input_step,6, 500)

# Define encoder architecture
inputs = Input(shape=input_shape)
x = Conv2D(128,3, activation='relu', padding='same' )(inputs)

x = MaxPooling2D((1,2), padding='same')(x)

x = Conv2D(64,3, activation='relu', padding='same')(x)

x = MaxPooling2D((1,2), padding='same')(x)
x = Conv2D(32,3, activation='relu', padding='same')(x)

encoded2 = Reshape((input_step,64))(x)



# Define decoder architecture
x = Reshape((input_step,1,64))(x)
x = Conv2D(64, (1,1), activation='relu', padding='same')(x)
x = UpSampling2D((1,2))(x)
x = Conv2D(100, (1,1), activation='relu', padding='same')(x)
x = UpSampling2D((1,2))(x)
x = Conv2D(128, (1,1), activation='relu', padding='same')(x)
x=Conv2D(252, (1,1), activation='sigmoid')(x)
x=Reshape((input_step,6,168))(x)
x=Dense(500, activation='linear')(x)
x=Reshape((input_step,6,500))(x)

decoded2 = Conv2D(500, 3, activation='sigmoid', padding='same')(x)

# Define autoencoder model
autoencoder2 = Model(inputs=inputs, outputs=decoded2)

# Compile model
autoencoder2.compile(optimizer='adam', loss='mae',metrics=['mae', 'mse',tf.keras.metrics.RootMeanSquaredError()])

# Print model summary
print(autoencoder2.summary())


Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 25, 6, 500)]      0         
                                                                 
 conv2d_16 (Conv2D)          (None, 25, 6, 128)        576128    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 25, 3, 128)       0         
 2D)                                                             
                                                                 
 conv2d_17 (Conv2D)          (None, 25, 3, 64)         73792     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 25, 2, 64)        0         
 2D)                                                             
                                                                 
 conv2d_18 (Conv2D)          (None, 25, 2, 32)         184

In [50]:
# Train the model
opt = Adam(learning_rate=0.0001)
autoencoder.compile(optimizer=opt, loss='mae',metrics=['mae', 'mse',tf.keras.metrics.RootMeanSquaredError()])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
autoencoder2.fit(X1, X1, epochs=300,batch_size=64,
                validation_data=(X2, X2),callbacks=[early_stopping]
              )

Epoch 1/300
364/364 [==============================] - 184s 502ms/step - loss: 0.1029 - mae: 0.1029 - mse: 0.0296 - root_mean_squared_error: 0.1719 - val_loss: 0.1128 - val_mae: 0.1128 - val_mse: 0.0378 - val_root_mean_squared_error: 0.1945
Epoch 2/300
364/364 [==============================] - 176s 484ms/step - loss: 0.0936 - mae: 0.0936 - mse: 0.0271 - root_mean_squared_error: 0.1646 - val_loss: 0.1083 - val_mae: 0.1083 - val_mse: 0.0340 - val_root_mean_squared_error: 0.1845
Epoch 3/300
364/364 [==============================] - 202s 554ms/step - loss: 0.0906 - mae: 0.0906 - mse: 0.0247 - root_mean_squared_error: 0.1571 - val_loss: 0.1065 - val_mae: 0.1065 - val_mse: 0.0314 - val_root_mean_squared_error: 0.1771
Epoch 4/300
364/364 [==============================] - 214s 588ms/step - loss: 0.0887 - mae: 0.0887 - mse: 0.0233 - root_mean_squared_error: 0.1525 - val_loss: 0.1067 - val_mae: 0.1067 - val_mse: 0.0304 - val_root_mean_squared_error: 0.1744
Epoch 5/300
364/364 [===============

In [51]:
encoder_2 = Model(inputs, encoded2)
encoder_2.compile(optimizer='adam', loss='mae')

In [52]:
encoder_2.save("encoder_cnn_"+str(input_step)+".h5")